### 목표 1: 상황별 음향 데이터 3초 미만 제외

In [1]:
ls

AnnotatedAudioFiles/
AnnotatedAudioFiles_tmp/
KNN_binary.ipynb
KNN_event_classification.ipynb
KNN_predict.ipynb
KNN_situation_classification.ipynb
Untitled-Copy1.ipynb
Untitled.ipynb
Untitled1.ipynb
Untitled2.ipynb
Untitled3.ipynb
binary_classification.ipynb
convert_mp3_to_wav.ipynb
convert_wav/
convert_wav.ipynb
convert_wav_final.ipynb
crop_event.ipynb
crop_wav.ipynb
cut_file.wav
data/
data_preprocessing.ipynb
dataframe_add_file_name.ipynb
event_data_preprocesssing.csv
event_data_preprocesssing_final.csv
event_data_preprocesssing_jjin_real_final.csv
event_data_preprocesssing_real_final.csv
ffmpeg-4.4.1-amd64-static/
knn_binary_model.pkl
knn_model.pkl
model_classification1.ipynb
model_classification2.ipynb
raw_data.ipynb
same_crop_wav.ipynb
same_preprocessing.ipynb
same_sound_extract/
sound_extract/
star_pydub.ipynb
test.ipynb


In [33]:
import os

# Pandas
import pandas as pd

# Scikit learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.utils import class_weight

# Keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical


# Audio
import librosa
import librosa.display

# Plot
import matplotlib.pyplot as plt

# Utility
import glob
import numpy as np
from tqdm import tqdm
import IPython.display as ipd
from timeit import default_timer as timer

import soundfile as sf

In [23]:
# glob는 파일들의 리스트를 뽑을 때 사용하는데, 파일의 경로명을 이용해서 입맛대로 요리할 수 있답니다. 
# 위의 glob() 함수는 인자로 받은 패턴과 이름이 일치하는 모든 파일과 디렉터리의 리스트를 반환합니다. 
# Building our dataset
dataset = list()
for folder in glob.glob('sound_extract/'):
    list_sounds_path = glob.glob(folder+"//**")
    for elm in list_sounds_path:
        label = elm.split(".wav")[0][-5]
        if label == '-':
            label = elm.split(".wav")[0][-6]
        dataset.append({"file_name" : elm})
        pass
    pass
pass
dataset = pd.DataFrame(dataset)
dataset = shuffle(dataset, random_state = 42)

In [24]:
dataset.head()

,file_name
9490,sound_extract/3500_dropping.wav
8796,sound_extract/3609_footstep.wav
6264,sound_extract/13790_groan.wav
1662,sound_extract/14494_male_speech.wav
5674,sound_extract/14883_male_speech.wav


In [26]:
len(dataset)

15855

In [35]:
ls

AnnotatedAudioFiles/
AnnotatedAudioFiles_tmp/
KNN_binary.ipynb
KNN_event_classification.ipynb
KNN_predict.ipynb
KNN_situation_classification.ipynb
Untitled-Copy1.ipynb
Untitled.ipynb
Untitled1.ipynb
Untitled2.ipynb
Untitled3.ipynb
binary_classification.ipynb
convert_mp3_to_wav.ipynb
convert_wav/
convert_wav.ipynb
convert_wav_final.ipynb
crop_event.ipynb
crop_wav.ipynb
cut_file.wav
data/
data_preprocessing.ipynb
dataframe_add_file_name.ipynb
event_data_preprocesssing.csv
event_data_preprocesssing_final.csv
event_data_preprocesssing_jjin_real_final.csv
event_data_preprocesssing_real_final.csv
ffmpeg-4.4.1-amd64-static/
knn_binary_model.pkl
knn_model.pkl
model_classification1.ipynb
model_classification2.ipynb
raw_data.ipynb
same_crop_wav.ipynb
same_preprocessing.ipynb
same_sound_extract/
sound_extract/
star_pydub.ipynb
test.ipynb


In [42]:
# 3초 이상인 음향 파일만 지정된 경로에 저장

def filter_audio(file_name):
    y, sr = librosa.load(file_name)
    time = librosa.get_duration(y=y, sr=sr)
    name = file_name.split('/')[1]
    if time >= 3.0:
        sf.write(f'same_sound_extract/save/{name}', y, sr)
        print(name, time)

In [43]:
for i in range(len(dataset)):
    filter_audio(dataset.file_name.iloc[i])

14494_male_speech.wav 3.8076190476190477
14883_male_speech.wav 7.1433560090702946
15068_male_speech.wav 8.747619047619047
6233_water.wav 3.1690249433106574
15715_sob.wav 29.327301587301587
5144_yelling.wav 3.4923356009070297
11417_footstep.wav 3.592607709750567
8320_male_speech.wav 4.246893424036281
2271_boom.wav 6.494648526077097
6721_groan.wav 24.401315192743763
9030_sob.wav 29.327301587301587
637_male_speech.wav 12.930249433106576
8011_male_speech.wav 8.024897959183674
2877_dropping.wav 5.300498866213152
5715_footstep.wav 3.546621315192744
13682_boom.wav 4.097732426303855
11251_water.wav 3.9213151927437644
15016_male_speech.wav 18.09015873015873
15206_sob.wav 10.124897959183674
8713_sob.wav 6.196417233560091
15587_sob.wav 18.289569160997733
8630_sob.wav 10.582403628117914
15991_yelling.wav 3.028027210884354
14347_male_speech.wav 6.143492063492063
15100_male_speech.wav 3.7835827664399093
15563_sob.wav 7.728843537414966
14957_male_speech.wav 9.834603174603174
14875_male_speech.wav 14.

### 목표 2: 데이터프레임 만들기

In [60]:
time_list = []
name_list = []
for i in range(len(dataset)):
    file_name = dataset.file_name.iloc[i]
    y, sr = librosa.load(file_name)
    time = librosa.get_duration(y=y, sr=sr)
    name = dataset.file_name.iloc[i].split('/')[1]
    if time >= 3.0:
        time_list.append(time)
        name_list.append(name)

In [61]:
print(len(time_list))
print(len(name_list))

4422
4422


In [63]:
df = pd.DataFrame({'file_name': name_list, 'time': time_list})
df

,file_name,time
0,14494_male_speech.wav,3.807619
1,14883_male_speech.wav,7.143356
2,15068_male_speech.wav,8.747619
3,6233_water.wav,3.169025
4,15715_sob.wav,29.327302
...,...,...
4417,15330_sob.wav,4.769841
4418,6761_groan.wav,4.974331
4419,9055_sob.wav,7.203764
4420,8283_male_speech.wav,3.102902


### 목표 3: 음향 데이터 3초로 자르기

In [44]:
ls

AnnotatedAudioFiles/
AnnotatedAudioFiles_tmp/
KNN_binary.ipynb
KNN_event_classification.ipynb
KNN_predict.ipynb
KNN_situation_classification.ipynb
Untitled-Copy1.ipynb
Untitled.ipynb
Untitled1.ipynb
Untitled2.ipynb
Untitled3.ipynb
binary_classification.ipynb
convert_mp3_to_wav.ipynb
convert_wav/
convert_wav.ipynb
convert_wav_final.ipynb
crop_event.ipynb
crop_wav.ipynb
cut_file.wav
data/
data_preprocessing.ipynb
dataframe_add_file_name.ipynb
event_data_preprocesssing.csv
event_data_preprocesssing_final.csv
event_data_preprocesssing_jjin_real_final.csv
event_data_preprocesssing_real_final.csv
ffmpeg-4.4.1-amd64-static/
knn_binary_model.pkl
knn_model.pkl
model_classification1.ipynb
model_classification2.ipynb
raw_data.ipynb
same_crop_wav.ipynb
same_preprocessing.ipynb
same_sound_extract/
sound_extract/
star_pydub.ipynb
test.ipynb


In [48]:
train_path = "same_sound_extract/save/"
train_file_list = os.listdir(train_path)
train_file_names = [file for file in train_file_list if file.endswith(".wav")]
train_file_names.sort(key=lambda x: (x.partition('.')[0]))
print(train_file_names)

['0_footstep.wav', '10007_door.wav', '10071_sob.wav', '10093_door.wav', '10137_door.wav', '10146_male_speech.wav', '1019_male_speech.wav', '10208_male_speech.wav', '10230_male_speech.wav', '10253_footstep.wav', '10260_boom.wav', '10268_appeal.wav', '10282_footstep.wav', '102_sob.wav', '10322_sob.wav', '10388_footstep.wav', '10395_footstep.wav', '103_footstep.wav', '10404_male_speech.wav', '10422_male_speech.wav', '1042_footstep.wav', '10430_appeal.wav', '10432_appeal.wav', '10439_appeal.wav', '10448_appeal.wav', '10449_male_speech.wav', '10476_sob.wav', '10477_footstep.wav', '10492_sob.wav', '1049_boom.wav', '10501_slapping.wav', '10511_groan.wav', '10526_dropping.wav', '10527_yelling.wav', '10528_sob.wav', '10537_slapping.wav', '10544_yelling.wav', '10567_sob.wav', '1057_appeal.wav', '105_groan.wav', '10659_male_speech.wav', '10664_groan.wav', '1071_footstep.wav', '10803_cutter.wav', '10824_door.wav', '10835_footstep.wav', '10894_door.wav', '108_groan.wav', '10913_footstep.wav', '1093

In [69]:
j = len(df)
j

4422

In [70]:
for i in range(j):
    if df.iloc[i]['file_name'] not in train_file_names:
        print(df.iloc[i]['file_name'])
    name = df.iloc[i]['file_name']
    file_path = 'same_sound_extract/save/' + df.iloc[i]['file_name']
    x, sr = sf.read(file_path, always_2d=True)
    start = int(0 * sr)
    end = int(3 * sr)
    y = x[start : end]
    if start==end:
        print(df.iloc[i]['file_name'])
        df.iloc[i] = None
        continue
    sf.write(f'same_sound_extract/trim/{name}', y, sr)

In [ ]:
# for i in range(j):
#     if df.iloc[i]['file_name'] not in train_file_names:
#         print(df.iloc[i]['file_name'])
#     file_path = 'same_sound_extract/save/' + df.iloc[i]['file_name']
#     x, sr = sf.read(file_path, always_2d=True)
#     start = int(df.iloc[i]['from'] * sr)
#     end = int(df.iloc[i]['to'] * sr)
#     y = x[start : end]
#     if start==end:
#         print(df.iloc[i]['file_name'])
#         df.iloc[i] = None
#         continue
#     event = df.iloc[i]['event']
#     sf.write(f'same_sound_extract/trim/{i}_{event}.wav', y, sr)